<a href="https://colab.research.google.com/github/Adithya707/Parallel_Programing/blob/main/Cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!apt-get --purge -y remove 'cuda*'
!apt-get --purge -y remove 'nvidia*'
!apt autoremove -y
!apt-get clean
!apt update -qq;
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.1.168-1_amd64.deb;
!dpkg -i cuda-repo-ubuntu1804_10.1.168-1_amd64.deb;
!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-10-1=10.1.168-1;

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'cuda-cusparse-cross-qnx-10-0' for glob 'cuda*'
Note, selecting 'cuda-toolkit-11-3-config-common' for glob 'cuda*'
Note, selecting 'cuda-nsight-compute-11-0' for glob 'cuda*'
Note, selecting 'cuda-nsight-compute-11-1' for glob 'cuda*'
Note, selecting 'cuda-nsight-compute-11-2' for glob 'cuda*'
Note, selecting 'cuda-nsight-compute-11-3' for glob 'cuda*'
Note, selecting 'cuda-nsight-compute-11-4' for glob 'cuda*'
Note, selecting 'cuda-nsight-compute-11-5' for glob 'cuda*'
Note, selecting 'cuda-nvprof-11-0' for glob 'cuda*'
Note, selecting 'cuda-nvprof-11-1' for glob 'cuda*'
Note, selecting 'cuda-nvprof-11-2' for glob 'cuda*'
Note, selecting 'cuda-nvprof-11-3' for glob 'cuda*'
Note, selecting 'cuda-nvprof-11-4' for glob 'cuda*'
Note, selecting 'cuda-nvprof-11-5' for glob 'cuda*'
Note, selecting 'cuda-npp-cross-qnx-10-0' for glob 'cuda*'
Note, selecting 'cuda-nvprune-10-0' for gl

In [1]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-b5buvhmy
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-b5buvhmy
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=df00486b7503cdc524c8cadaac1b5bc41a7789a915a535255f50232883ec9c15
  Stored in directory: /tmp/pip-ephem-wheel-cache-gffufxiz/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [11]:
%%cu
#include <iostream>
int main() {
    std::cout << "Hello world\n";
    return 0;
}

Hello world



In [12]:
%%cu
#include <stdio.h>
#define N  64
inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}
__global__ void matrixMulGPU( int * a, int * b, int * c )
{
  /*
   * Build out this kernel.
   */
    int row = threadIdx.y + blockIdx.y * blockDim.y;
    int col = threadIdx.x + blockIdx.x * blockDim.x;
    
    int val = 0;
    if (row < N && col < N) {
      for (int i = 0; i < N; ++i) {
         val += a[row * N + i] * b[i * N + col];
       }
    
      c[row * N + col] = val;
    }
}
 /*
 * This CPU function already works, and will run to create a solution matrix
 * against which to verify your work building out the matrixMulGPU kernel.
 */
void matrixMulCPU( int * a, int * b, int * c )
{
  int val = 0;for( int row = 0; row < N; ++row )
    for( int col = 0; col < N; ++col )
    {
      val = 0;
      for ( int k = 0; k < N; ++k )
        val += a[row * N + k] * b[k * N + col];
      c[row * N + col] = val;
    }
}int main()
{
  int *a, *b, *c_cpu, *c_gpu; // Allocate a solution matrix for both the CPU and the GPU operationsint 
  int size = N * N * sizeof (int); // Number of bytes of an N x N matrix// Allocate memory
  cudaMallocManaged (&a, size);
  cudaMallocManaged (&b, size);
  cudaMallocManaged (&c_cpu, size);
  cudaMallocManaged (&c_gpu, size);// Initialize memory; create 2D matrices
  for( int row = 0; row < N; ++row )
    for( int col = 0; col < N; ++col )
    {
      a[row*N + col] = row;
      b[row*N + col] = col+2;
      c_cpu[row*N + col] = 0;
      c_gpu[row*N + col] = 0;
    }
   /*
   * Assign `threads_per_block` and `number_of_blocks` 2D values
   * that can be used in matrixMulGPU above.
   */
  dim3 threads_per_block(32, 32, 1);
  dim3 number_of_blocks(N / threads_per_block.x + 1, N / threads_per_block.y + 1, 1);
  matrixMulGPU <<< number_of_blocks, threads_per_block >>> ( a, b, c_gpu );checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
  checkCudaErr(cudaGetLastError(), "GPU");// Call the CPU version to check our work
  matrixMulCPU( a, b, c_cpu );// Compare the two answers to make sure they are equal
  bool error = false;
  for( int row = 0; row < N && !error; ++row )
    for( int col = 0; col < N && !error; ++col )
      if (c_cpu[row * N + col] != c_gpu[row * N + col])
      {
        printf("FOUND ERROR at c[%d][%d]\n", row, col);
        error = true;
        break;
      }
  if (!error)
    printf("Success!\n");// Free all our allocated memory
  cudaFree(a);
  cudaFree(b);
  cudaFree( c_cpu );
  cudaFree( c_gpu );
}

Success!



In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cuda.cu to cuda.cu


In [ ]:
!ls

a.out  cuda  cuda.cu  sample_data


In [16]:
%%cu
#include <stdio.h>

__global__ void add(int *a, int *b, int *c, int n)
{
    int i = threadIdx.x;
    if (i % 2 == 0)
    {
        a[i] = b[i + 1];
        a[i + 1] = b[i];
    }

    else
    {
        a[i] = b[i - 1];
        a[i - 1] = b[i];
    }
    c[i] = a[i];
}

int main(void)
{

    int *d_a, *d_b, *d_c, n;
    int size = sizeof(int);

    printf("Enter size of array \n");
    scanf("%d", &n);
    int a[n * 2], b[n * 2], c[n * 2], i;
    cudaMalloc((void **)&d_a, (size * n));
    cudaMalloc((void **)&d_b, (size * n));
    cudaMalloc((void **)&d_c, (size * n));
    printf("Enter value for a and b\n");
    for (i = 0; i < n; i++)
    {
        printf("Enter valur for a[%d] ", i);
        scanf("%d", &a[i]);
        b[i] = a[i];
    }
    printf("\n");
    cudaMemcpy(d_a, &a, (size * n), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, &b, (size * n), cudaMemcpyHostToDevice);
    add<<<1,n>>>(d_a, d_b, d_c, n);
    cudaMemcpy(&c, d_c, (size * n), cudaMemcpyDeviceToHost);
    printf("Resultant array \n");
    for (int i = 0; i < n; i++)
        printf("%d ", c[i]);
    printf("\n");
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    return 0;
}